In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Substance_Use_Disorder"
cohort = "GSE94399"

# Input paths
in_trait_dir = "../../input/GEO/Substance_Use_Disorder"
in_cohort_dir = "../../input/GEO/Substance_Use_Disorder/GSE94399"

# Output paths
out_data_file = "../../output/preprocess/Substance_Use_Disorder/GSE94399.csv"
out_gene_data_file = "../../output/preprocess/Substance_Use_Disorder/gene_data/GSE94399.csv"
out_clinical_data_file = "../../output/preprocess/Substance_Use_Disorder/clinical_data/GSE94399.csv"
json_path = "../../output/preprocess/Substance_Use_Disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptome profiles of liver biopsy tissues from sever alcoholic hepatitis patients (validation cohort, Brussels)"
!Series_summary	"Corticosteroids are the current standard of care to improve short_term mortality in severe alcoholic hepatitis (AH), although nearly 40% of the patients do not respond and accurate pre_treatment predictors are lacking. We developed 123_gene prognostic score based on molecular and clinical variables before initiation of corticosteroids. Furthermore, The gene signature was implemented in an FDA_approved platform (NanoString), and verified for technical validity and prognostic capability. Here we demonstrated that a Nanostring_based gene expressoin risk classificatoin is useful to predict mortality in  patients with severe alcoholic hepatitis who were treated by corticosteroid"
!Series_overall_design	"Gene expression profiling of formalin-fixed paraffin-embedded liver biopsy tissues obtained at the time of enrollment.

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data
# specifically from liver biopsy tissues as mentioned in "Series_overall_design"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Looking at the Sample Characteristics Dictionary
# For trait (Substance Use Disorder): The dictionary shows outcome in row 1,
# which relates to alcoholic hepatitis (a substance use disorder from background info)
# For age and gender: Not explicitly available in the sample characteristics

# Trait is available in row 1 (outcome at 6 months)
trait_row = 1
# Age and gender are not available in the sample characteristics
age_row = None
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert outcome data to binary trait value for Substance Use Disorder"""
    if value is None or not isinstance(value, str):
        return None
    
    # Extract the value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary: 1 for negative outcome (dead/transplant), 0 for alive
    if "alive" in value.lower():
        return 0
    elif "dead" in value.lower() or "transplantation" in value.lower():
        return 1
    else:
        return None

def convert_age(value):
    """Convert age data to continuous values"""
    # Not used as age is not available
    return None

def convert_gender(value):
    """Convert gender data to binary values (0 for female, 1 for male)"""
    # Not used as gender is not available
    return None

# 3. Save Metadata
# Determine trait data availability and conduct initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (Only if trait_row is not None)
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    # The dictionary from the previous output has keys as rows and values as lists of characteristics
    sample_chars = {
        0: ['cohort: Validation cohort (BRAH)'], 
        1: ['outcome at 6 months: Alive', 'outcome at 6 months: Dead or liver transplantation']
    }
    
    # Build a matrix-like structure that mimics the expected clinical data format
    # First determine all sample IDs based on the data available in row 1
    sample_ids = []
    for i in range(len(sample_chars[1])):
        sample_ids.append(f"GSM{i+1}")
    
    # Create a dictionary to build our DataFrame
    data_dict = {'Sample': sample_ids}
    
    # Add the characteristic values for each row
    for row, values in sample_chars.items():
        # Check if there's one value per sample or just one value for all samples
        if len(values) == len(sample_ids):
            # One value per sample
            data_dict[row] = values
        else:
            # Repeat the same value for all samples or handle differently if needed
            # For simplicity, we'll just use the first value for all samples if there's a mismatch
            data_dict[row] = [values[0]] * len(sample_ids)
    
    # Create DataFrame
    clinical_data = pd.DataFrame(data_dict)
    clinical_data.set_index('Sample', inplace=True)
    
    # Use geo_select_clinical_features to extract features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    print("Preview of selected clinical features:")
    print(preview_df(selected_clinical_df))
    
    # Save the clinical features to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{0: [nan], 1: [1.0]}
Clinical data saved to ../../output/preprocess/Substance_Use_Disorder/clinical_data/GSE94399.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 58
Header line: "ID_REF"	"GSM2474751"	"GSM2474752"	"GSM2474753"	"GSM2474754"	"GSM2474755"	"GSM2474756"	"GSM2474757"	"GSM2474758"	"GSM2474759"	"GSM2474760"	"GSM2474761"	"GSM2474762"	"GSM2474763"	"GSM2474764"	"GSM2474765"	"GSM2474766"	"GSM2474767"	"GSM2474768"	"GSM2474769"	"GSM2474770"	"GSM2474771"	"GSM2474772"	"GSM2474773"	"GSM2474774"	"GSM2474775"	"GSM2474776"	"GSM2474777"	"GSM2474778"	"GSM2474779"	"GSM2474780"	"GSM2474781"	"GSM2474782"	"GSM2474783"	"GSM2474784"	"GSM2474785"	"GSM2474786"	"GSM2474787"	"GSM2474788"
First data line: "11715100_at"	14.61709571	13.42065958	19.6573113	22.48539724	52.65458339	29.87886953	15.64211321	21.14364386	37.30879312	14.78742501	17.08143895	16.14591018	13.26892829	12.68560441	13.18877976	13.66343127	15.88047304	14.94092906	11.28935481	12.45280671	10.92842639	9.324238654	18.42329466	5.363407605	18.08406026	41.03262025	30.92712002	20.42521934	16.11238155	11.77488985	15.73874147	11.7609501	15.12186329	28.66298794	12.07415117	13.500

Index(['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at',
       '11715104_s_at', '11715105_at', '11715106_x_at', '11715107_s_at',
       '11715108_x_at', '11715109_at', '11715110_at', '11715111_s_at',
       '11715112_at', '11715113_x_at', '11715114_x_at', '11715115_s_at',
       '11715116_s_at', '11715117_x_at', '11715118_s_at', '11715119_s_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Analyzing gene identifiers
# The IDs are in the format "11715100_at", which appears to be Affymetrix probe IDs 
# from an Affymetrix microarray chip rather than standard human gene symbols
# These probe IDs need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE94399
Line 6: !Series_title = Transcriptome profiles of liver biopsy tissues from sever alcoholic hepatitis patients (validation cohort, Brussels)
Line 7: !Series_geo_accession = GSE94399
Line 8: !Series_status = Public on Feb 18 2018
Line 9: !Series_submission_date = Feb 01 2017
Line 10: !Series_last_update_date = Mar 21 2019
Line 11: !Series_pubmed_id = 29158192
Line 12: !Series_summary = Corticosteroids are the current standard of care to improve short_term mortality in severe alcoholic hepatitis (AH), although nearly 40% of the patients do not respond and accurate pre_treatment predictors are lacking. We developed 123_gene prognostic score based on molecular and clinical variables before initiation o


Gene annotation preview:
{'ID': ['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at', '11715104_s_at'], 'GeneChip Array': ['Human Genome HG-U219 Array', 'Human Genome HG-U219 Array', 'Human Genome HG-U219 Array', 'Human Genome HG-U219 Array', 'Human Genome HG-U219 Array'], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['20-Aug-10', '20-Aug-10', '20-Aug-10', '20-Aug-10', '20-Aug-10'], 'Sequence Type': ['Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database'], 'Transcript ID(Array Design)': ['g21264570', 'g21264570', 'g21264570', 'g22748780', 'g30039713'], 'Target Description': ['g21264570 /TID=g21264570 /CNT=1 /FEA=FLmRNA /TIER=FL /STK=0 /DEF=g2126

### Step 6: Gene Identifier Mapping

In [7]:
# Identify which columns in the gene annotation dataframe correspond to gene identifiers and gene symbols
# From the previous output, we can see:
# - The gene expression data uses IDs like "11715100_at"
# - The gene annotation has an "ID" column with these probe IDs
# - The "Gene Symbol" column contains the gene symbols we want to map to

# 1. Extract the gene identifier and gene symbol columns for mapping
prob_col = 'ID'  # Column with probe IDs matching gene expression data
gene_col = 'Gene Symbol'  # Column with gene symbols

# 2. Get the gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Gene mapping preview (first 5 rows):")
print(preview_df(gene_mapping))

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"\nGene expression data after mapping (first 5 rows, 3 columns):")
print(gene_data.head(5)[gene_data.columns[:3]])

# 4. Get the shape of the resulting gene expression dataframe
print(f"\nShape of gene expression data: {gene_data.shape}")


Gene mapping preview (first 5 rows):
{'ID': ['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at', '11715104_s_at'], 'Gene': ['HIST1H3G', 'HIST1H3G', 'HIST1H3G', 'TNFAIP8L1', 'OTOP2']}

Gene expression data after mapping (first 5 rows, 3 columns):
       GSM2474751  GSM2474752  GSM2474753
Gene                                     
A1BG   235.239452  166.162065  139.648057
A1CF   357.729179  296.125841  233.119900
A2BP1   29.494727   34.639537   44.718189
A2LD1    4.900975    4.134265    4.364917
A2M    570.793025  799.633007  616.235048

Shape of gene expression data: (19521, 38)


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
print(f"Gene data shape before normalization: {gene_data.shape}")

# Normalize gene symbols using NCBI Gene database
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if trait data is available before proceeding with clinical data extraction
if trait_row is None:
    print("Trait row is None. Cannot extract trait information from clinical data.")
    # Create an empty dataframe for clinical features
    clinical_features = pd.DataFrame()
    
    # Create an empty dataframe for linked data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Trait data is not available
        is_biased=True,  # Not applicable but required
        df=pd.DataFrame(),  # Empty dataframe
        note=f"Dataset contains gene expression data but lacks clear trait indicators for {trait} status."
    )
    print("Data was determined to be unusable due to missing trait indicators and was not saved")
else:
    try:
        # Get the file paths for the matrix file to extract clinical data
        _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        
        # Get raw clinical data from the matrix file
        _, clinical_raw = get_background_and_clinical_data(matrix_file)
        
        # Verify clinical data structure
        print("Raw clinical data shape:", clinical_raw.shape)
        
        # Extract clinical features using the defined conversion functions
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print("Clinical features:")
        print(clinical_features)
        
        # Save clinical features to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
        # 3. Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, first 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 4. Handle missing values
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        is_trait_biased = False
        if len(cleaned_data) > 0:
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
            is_trait_biased = trait_biased
        else:
            print("No data remains after handling missing values.")
            is_trait_biased = True
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note=f"Dataset contains gene expression data for {trait} analysis."
        )
        
        # 7. Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
            
    except Exception as e:
        print(f"Error processing data: {e}")
        # Handle the error case by still recording cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,  # Mark as not available due to processing issues
            is_biased=True, 
            df=pd.DataFrame(),  # Empty dataframe
            note=f"Error processing data for {trait}: {str(e)}"
        )
        print("Data was determined to be unusable and was not saved")

Gene data shape before normalization: (19521, 38)


Gene data shape after normalization: (19298, 38)


Normalized gene data saved to ../../output/preprocess/Substance_Use_Disorder/gene_data/GSE94399.csv
Raw clinical data shape: (2, 39)
Clinical features:
                        GSM2474751  GSM2474752  GSM2474753  GSM2474754  \
Substance_Use_Disorder         0.0         0.0         0.0         1.0   

                        GSM2474755  GSM2474756  GSM2474757  GSM2474758  \
Substance_Use_Disorder         0.0         1.0         1.0         1.0   

                        GSM2474759  GSM2474760  ...  GSM2474779  GSM2474780  \
Substance_Use_Disorder         1.0         1.0  ...         1.0         1.0   

                        GSM2474781  GSM2474782  GSM2474783  GSM2474784  \
Substance_Use_Disorder         0.0         0.0         0.0         1.0   

                        GSM2474785  GSM2474786  GSM2474787  GSM2474788  
Substance_Use_Disorder         0.0         0.0         1.0         1.0  

[1 rows x 38 columns]
Clinical features saved to ../../output/preprocess/Substance_Use_Disorder

Data shape after handling missing values: (38, 19299)
For the feature 'Substance_Use_Disorder', the least common label is '1.0' with 15 occurrences. This represents 39.47% of the dataset.
The distribution of the feature 'Substance_Use_Disorder' in this dataset is fine.



Linked data saved to ../../output/preprocess/Substance_Use_Disorder/GSE94399.csv
